<h1>Module 1 Key:</h1>

Most any method is acceptable here, so long as it has the desired output. This is meant to be an exploratory exercise so any approach is reasonable.

In [3]:
import numpy as np
# Read in the data file from a CSV with delimiter of ';', and each value will be read as a string
wellData = np.genfromtxt("Wells.csv", delimiter=';', dtype=str)
print(wellData)

def api10GenKey(data):
#     Read in the data
    apis, api10 = data.T[1][1:], []
#     Loop over the original APIs
    for api in apis:
#         Check for hyphens
        if '-' in api:
            ind = api.index('-')
            api = str(api[:ind]) + str(api[ind+1:])
#         Check if the API begins with a 5
        if len(api) > 8 and len(api) < 10:
            if api[0] == '5':
                api = "0" + str(api)
                continue
            else: 
                print("Following API Unexpected:" + str(api))
#         Check if already API10
        elif len(api) == 10 and api[0:2] == "05":
            continue
        api = "05" + str(api)
        api10.append(api)
#     Write the output to a file.
    np.savetxt('api10s.csv', api10, delimiter=',',fmt='%s')
    return api10

api10GenKey(wellData)

[['FID' 'API' 'Operator' ..., 'Deviation' 'Field_Code' 'Field_Name']
 ['0' '10613700' 'BAYSWATER EXPLORATION & PRODUCTION LLC' ..., 'Drifted'
  '33399' 'GUNBARREL']
 ['1' '510621300' 'GREAT WESTERN OPERATING COMPANY LLC' ..., 'Drifted'
  '73760' 'RIMROCK']
 ..., 
 ['98' '10969600' 'COLORADO INTERSTATE GAS COMPANY LLC' ..., 'Horizontal'
  '83000' 'TOTEM']
 ['99' '10970300' 'COLORADO INTERSTATE GAS COMPANY LLC' ..., 'Horizontal'
  '83000' 'TOTEM']
 ['100' '10970400' 'COLORADO INTERSTATE GAS COMPANY LLC' ..., 'Horizontal'
  '83000' 'TOTEM']]


['0510613700',
 '0510623101',
 '0510624800',
 '0510624801',
 '0510628100',
 '0510628501',
 '0510637701',
 '0510642000',
 '0510643800',
 '0510645100',
 '0510649200',
 '0510662800',
 '0510670300',
 '0510674700',
 '0510679300',
 '0510698400',
 '0510698701',
 '0510702300',
 '0510705300',
 '0510719900',
 '0510720100',
 '0510740900',
 '0510757800',
 '0510785600',
 '0510802300',
 '0510805900',
 '0510814200',
 '0510826600',
 '0510866800',
 '0510872000',
 '0510876700',
 '0510876800',
 '0510881000',
 '0510882900',
 '0510883100',
 '0510883900',
 '0510890200',
 '0510891900',
 '0510898200',
 '0510899700',
 '0510902300',
 '0510907300',
 '0510910300',
 '0510912000',
 '0510913200',
 '0510919200',
 '0510919300',
 '0510927400',
 '0510927500',
 '0510929200',
 '0510932900',
 '0510933000',
 '0510933100',
 '0510933200',
 '0510936200',
 '0510936300',
 '0510936500',
 '0510936600',
 '0510942500',
 '0510942600',
 '0510945700',
 '0510946800',
 '0510950600',
 '0510950900',
 '0510951700',
 '0510951800',
 '05109527

<h1>Module 2 Key:</h1>

This task should be relatively straightforward and is meant to be an introduction to working with the Python Window interface. If the users are able to use ArcPy in Jupyter notebooks then submission via notebook works, as does a separate text file with the code copied from the Python Window that ran within ArcMap. 

In [6]:
import arcpy
import os

# Establish the mapping document
mxd = arcpy.mapping.MapDocument(r"C:\Users\Tannor - Desktop\Documents\ArcEnv\Python for ArcGis\CO_Wells\CO_Wells.mxd")

# define the default directory/workspace
default_dir = mxd.filePath.rsplit('\\', 1)[0]

# Make the temporary planned horizontals layer
arcpy.MakeFeatureLayer_management(default_dir + "\\" + "Directional_Lines.shp", "Planned_Directional_Lines", "Dir_Status = 'Planned'")

# Create the new shapefile via the temporary layer
arcpy.FeatureClassToShapefile_conversion("Planned_Directional_Lines", default_dir)
del mxd

<h1>Module 3 Key:</h1>

This assignment is more complicated than the previous ones and may require more assistance for successful submissions. The below code utilizes a single absolute path which will need to be modified for the mapping document, but the rest of the below code should run fine. Common issues that users may encounter are as follows:

#### Invalid SQL Expression
The SQL expressions passed in for the MakeFeatureLayer_management can be extremely picky, and may result in a 000358 error. The first point of troubleshooting should be to examine the SQL query and make sure that the entire query is encapsulated in one quotation style, with the value to search for in a different style (double vs single quotes).

In [7]:
import arcpy
import os

# Establish the mapping document
mxd = arcpy.mapping.MapDocument(r"C:\Users\Tannor - Desktop\Documents\ArcEnv\Python for ArcGis\CO_Wells\CO_Wells.mxd")

# Establish the working environment
default_dir = mxd.filePath[:-12] 
arcpy.env.overwriteOutput = True

# Create the temporary TR layer
arcpy.MakeFeatureLayer_management("CO_Townships.shp", "Target_TR", "TWNSHPLAB = '10N 58W' OR TWNSHPLAB = '10N 59W' OR TWNSHPLAB = '10N 60W'")

# Clip the Total Directional Lines shp to only those in the TR
arcpy.Clip_analysis("Directional_Lines.shp", "Target_TR", arcpy.env.workspace+"\\TR_Directionals.shp")

# Add the field to the attribute table
newFieldName = "API10"
arcpy.AddField_management(default_dir + "\\TR_Directionals.shp", newFieldName, "TEXT", "", "", "", "Custom API10", "NON_NULLABLE") 

# Create Search and Update cursors to comb through the data and create APIS
# Make empty list for storing new APIS
APIs = []

# Establish Feature class name and field name
fc = arcpy.env.workspace + "\\TR_Directionals.shp"
fieldName = 'API'

# Index to keep track of new API location in relation to attribute table  
index = 0

# Use Search cursor to locate APIS and determine how each needs to be modified
with arcpy.da.SearchCursor(fc, fieldName) as cursor:
    for row in cursor:
        if(len(row[0]) == 10):
            APIs.append("05" + str(row[0][:-2]))
        else:
            APIs.append("05" + str(row[0]))
    del row
    del cursor
    
# Use Update cursor to locate each row and assign the corresponding API10
with arcpy.da.UpdateCursor(fc, newFieldName) as cursor:
    for row in cursor:
        row[0] = APIs[index]
        index += 1
        cursor.updateRow(row)
    del row
    del cursor

<h1>Module 4 Key:</h1>

This project represents a culmination of all the previous modules, and is a rather hefty submission. Below you will find a well annotated Python Toolbox which accomplishes the tasks set out in Module 4, along with a standalone script which can be run via commandline to issue all tools in sequence. 

In [ ]:
#  ===== Key for dbuffer method ========== 
def dBuffer(self, parameters):
#         Read in your data and establish variable names
        data = parameters[0]
        dist = parameters[1].valueAsText
        desc = arcpy.Describe(data)
        fcName = desc.name
        # NOTE: Current is used in this case as this tool is intended to be run only when the Map is open. 
        # An absolute path would be necessary if planning to run while the map was closed.
        mxd = arcpy.mapping.MapDocument("CURRENT")
        df = mxd.activeDataFrame
        arcpy.env.overwriteOutput = True
        default_dir = mxd.filePath[:-12]
        arcpy.Buffer_analysis(data.value, default_dir + "\\Buffered_" + fcName + ".shp", dist + " Feet", method="GEODESIC")
        arcpy.mapping.AddLayer(df, arcpy.mapping.Layer(default_dir + "\\Buffered_" + fcName))
        return 

In [8]:
# ======= Key: Toolbox =============
import sys
import os
import arcpy

class Toolbox(object):
    def __init__(self):
        self.label = "Horizontal Prep"
        self.alias = "hp"

        # List of tool classes associated with this toolbox
        self.tools = [dClip, dBuffer, API10Gen, intersector]

class dClip(object):
    def __init__(self):
        """Define the tool (tool name is the name of the class)."""
        self.label = "Directional Clip"
        self.description = "This tool is intended to clip directional well sticks to a given county."
        self.canRunInBackground = False

    def getParameterInfo(self):
        """Define parameter definitions"""
        param0 = arcpy.Parameter(
            displayName   = "Directional Well Feature Class",
            name          = "in_feature",
            datatype      = "DEFeatureClass",
            parameterType = "Required",
            direction     = "Input"
            )
        param1 = arcpy.Parameter(
            displayName   = "County Feature Class",
            name          = "clip_feature",
            datatype      = "DEFeatureClass",
            parameterType = "Required",
            direction     = "Input"
        )
        param2 = arcpy.Parameter(
            displayName   = "County Name",
            name          = "county_name",
            datatype      = "GPString",
            parameterType = "Required",
            direction     = "Input"
        )
        params = [param0, param1, param2]
        return params

    def isLicensed(self):
        """Set whether tool is licensed to execute."""
        return True


    def updateParameters(self, parameters):
        return

    def updateMessages(self, parameters):
#         Make sure the parameter is set before evaluating it
        if (parameters[0].value) is not None:
#         Access the shape from the file via describe
            dirFc = parameters[0].value
            shape = arcpy.Describe(dirFc).shapeType.upper()
#         Check to see if the shapefile contains polylines (horizontal wells) or points (vertical wells)
            if shape != 'POLYLINE':
                parameters[0].setWarningMessage(str("The source geometry does not contain Polylines. Please make sure that this feature contains proper geometry for this action"))
                errors = parameters[0].hasWarning()
#         Make sure the parameter is set before evaluating it
        if (parameters[1].value) is not None:
#         Check to see if the Counties fc contains polygons
            countFc = parameters[1]
            shape = arcpy.Describe(countFc).shapeType.upper()
            if shape != 'POLYGON':
                parameters[1].setWarningMessage(str("The source geometry does not contain Polygons. Please make sure that this feature contains proper geometry for this action"))
                errors = parameters[1].hasWarning()

#         Make sure the parameter is set before evaluating it
        if (parameters[2].value) is not None:
#         Check if the county name is actually in the shapefile
            countName = parameters[2].value
            contains = False
            with arcpy.da.SearchCursor(countFc.value, "County") as cursor:
                for row in cursor:
                    if(row[0] == countName):
                        contains = True
                    del row
                del cursor
            if contains == False:
                parameters[2].setErrorMessage(str("The provided county name is not present in the County shapefile."))
                errors = parameters[2].hasWarning()
        return

    def clipper(self, parameters):
#         Allow overwrites and establish environment
        arcpy.env.overwriteOutput = True
#         NOTE: Current is used in this case as this tool is intended to be run only when the Map is open. An absolute path would be necessary if planning to run while the map was closed.
        default_dir = arcpy.mapping.MapDocument("CURRENT").filePath
        default_dir = default_dir.rsplit('\\', 1)[0]
#         Access values from the passed parameters
        dirFc = parameters[0].value
        countFc = parameters[1].value
        countName = parameters[2].value
        dirFcName = arcpy.Describe(dirFc).name
#         Create the temporary County layer for the clipping
        arcpy.MakeFeatureLayer_management(countFc, countName + "_Temp", "County = '" + countName + "'")
#         Create the temporary directional lines layer containing actual wells
        arcpy.MakeFeatureLayer_management(dirFc, "Actual_" + dirFcName, "Dir_status = 'Actual'")
#         Perform the clip
        arcpy.Clip_analysis("Actual_" + dirFcName, countName+"_Temp", arcpy.env.workspace+"\\" + countName + "_Actual_" + dirFcName + ".shp")
        return

    def execute(self, parameters, messages):
        self.clipper(parameters)
        return

class dBuffer(object):
    def __init__(self):
        """Define the tool (tool name is the name of the class)."""
        self.label = "Buffer Tool"
        self.description = ""
        self.canRunInBackground = False

    def getParameterInfo(self):
        """Define parameter definitions"""
        param0 = arcpy.Parameter(
            displayName   = "Input Feature Class",
            name          = "in_Feature",
            datatype      = "DEFeatureClass", #DEFeatureClass represents a class where the datatype is consistent, i.e. polygons
            parameterType = "Required",
            direction     = "Input"
        )
        param1 = arcpy.Parameter(
            displayName   = "Buffer Distance",
            name          = "value",
            datatype      = "GPLong", #DEFeatureClass represents a class where the datatype is consistent, i.e. polygons
            parameterType = "Required",
            direction     = "Input"
        )
        params = [param0, param1]
        return params

    def isLicensed(self):
        """Set whether tool is licensed to execute."""
        return True
    
    def updateParameters(self, parameters):
#         Check to see if the shapefile contains polylines (horizontal wells) or points (vertical wells)
        inFile = parameters[0].value
        shape = arcpy.Describe(inFile).shapeType.upper()
#         If it contains polylines (sticks) then set the default to 200, else 100
        if shape == 'POLYLINE':
            parameters[1].value = 200
        else:
            parameters[1].value = 100
        return

    def updateMessages(self, parameters):
#         Check if default values are being used and issue a warning 
        if parameters[1].value == 100 or parameters[1].value == 200:
            parameters[1].setWarningMessage(str("The value shown is a default value. Please ensure that this applies to your particular usecase."))
            errors = parameters[1].hasWarning()
        return

    def dBuffer(self, parameters):
#         Set values from parameters
        data = parameters[0]
        dist = parameters[1].valueAsText
        desc = arcpy.Describe(data)
        fcName = desc.name
#         NOTE: Current is used in this case as this tool is intended to be run only when the Map is open. An absolute path would be necessary if planning to run while the map was closed.
#         Set the environment and overwrite access
        mxd = arcpy.mapping.MapDocument("CURRENT")
        df = mxd.activeDataFrame
        default_dir = mxd.filePath[:-12]
        arcpy.env.workspace = default_dir
#         Issue buffer command 
        arcpy.Buffer_analysis(data.value, default_dir + "\\Buffered_" + fcName, dist + " Feet", method="GEODESIC")
#         Add the layer to the map
        arcpy.mapping.AddLayer(df, arcpy.mapping.Layer(default_dir + "\\Buffered_" + fcName))
        return 

    def execute(self, parameters, messages):
        """The source code of the tool."""
        self.dBuffer(parameters)
        return

class API10Gen(object):
    def __init__(self):
        """Define the tool (tool name is the name of the class)."""
        self.label = "API10Gen"
        self.description = "This tool is intended to generate 10 digit APIs for wells in Colorado (State code 05)."
        self.canRunInBackground = False

    def getParameterInfo(self):
        """Define parameter definitions"""
        param0 = arcpy.Parameter(
            displayName   = "Well Feature Class",
            name          = "in_feature",
            datatype      = "DEFeatureClass",
            parameterType = "Required",
            direction     = "Input"
            )
        param1 = arcpy.Parameter(
            displayName   = "API Attribute Name",
            name          = "api_name",
            datatype      = "GPString",
            parameterType = "Required",
            direction     = "Input"
        )
        params = [param0, param1]
        return params

    def isLicensed(self):
        """Set whether tool is licensed to execute."""
        return True


    def updateParameters(self, parameters):
        return

    def updateMessages(self, parameters):
#         Check to see if the parameter is set 
        if (parameters[1].value) is not None:
#         Check to see if the fc contains the api column specified
            fc = parameters[0].value
            apiName = parameters[1].valueAsText
            if len(arcpy.ListFields(fc, apiName)) == 0:
                parameters[1].setWarningMessage(str("The attribute table does not conain the field specified."))
                errors = parameters[1].hasWarning()
        return

    def API10Gen(self, parameters):
#         Set the environment and allow overwrite
        arcpy.env.overwriteOutput = True
#         NOTE: Current is used in this case as this tool is intended to be run only when the Map is open. An absolute path would be necessary if planning to run while the map was closed.
        mxd = arcpy.mapping.MapDocument("CURRENT")
        default_dir = mxd.filePath
        default_dir = default_dir.rsplit('\\', 1)[0]
#         Establish values and variables from the parameters
        newFieldName = "API10"
        fc = parameters[0].value
        desc = arcpy.Describe(fc)
        fcName = desc.name
        cField = parameters[1].valueAsText
#         Create the field within the target fc attribute table
        arcpy.AddField_management(arcpy.env.workspace + "\\" + fcName, newFieldName, "TEXT", "", "", "", "Custom API10", "NON_NULLABLE")
#         Create an empty list and index variable for assigning values
        index = 0     
        APIs = []
#         Search through the existing APIs and determine the final API10 per row
        with arcpy.da.SearchCursor(fc, cField) as cursor:
            for row in cursor:
                if(len(row[0]) == 10):
                    APIs.append("05" + str(row[0][:-2]))
                else:
                    APIs.append("05" + str(row[0]))
                del row
        del cursor
#         Update the new fc attribute table with the new API10s
        with arcpy.da.UpdateCursor(fc, newFieldName) as cursor:
            for row in cursor:
                row[0] = APIs[index]
                index += 1
                cursor.updateRow(row)
        del row
        del cursor
        return

    def execute(self, parameters, messages):
        self.API10Gen(parameters)
        return

class intersector(object):
    def __init__(self):
        """Define the tool (tool name is the name of the class)."""
        self.label = "Intersector"
        self.description = "This tool is intended to intersect buffered horizontal well sticks with PLSS feature class information."
        self.canRunInBackground = False

    def getParameterInfo(self):
        """Define parameter definitions"""
        param0 = arcpy.Parameter(
            displayName   = "Well Feature Class",
            name          = "in_feature",
            datatype      = "DEFeatureClass",
            parameterType = "Required",
            direction     = "Input"
            )
        param1 = arcpy.Parameter(
            displayName   = "PLSS Feature Class",
            name          = "intersect_with",
            datatype      = "DEFeatureClass",
            parameterType = "Required",
            direction     = "Input"
        )
        param2 = arcpy.Parameter(
            displayName   = "Cluster Tolerance",
            name          = "ct",
            datatype      = "GPString",
            parameterType = "Required",
            direction     = "Input"
        )
        param3 = arcpy.Parameter(
            displayName   = "Output Type",
            name          = "out_type",
            datatype      = "GPString",
            parameterType = "Required",
            direction     = "Input"
        )
        param3.value = 'Input'
        params = [param0, param1, param2, param3]
        return params

    def isLicensed(self):
        """Set whether tool is licensed to execute."""
        return True


    def updateParameters(self, parameters):
        return

    def updateMessages(self, parameters):
        return

    def intersector(self, parameters):
#         Set the environment and allow overwrite access
#         NOTE: Current is used in this case as this tool is intended to be run only when the Map is open. An absolute path would be necessary if planning to run while the map was closed.
        mxd = arcpy.mapping.MapDocument("CURRENT")
        arcpy.env.overwriteOutput = True
        default_dir = mxd.filePath
        default_dir = default_dir.rsplit('\\', 1)[0]
#         Establish values from the parameters
        in_features = [parameters[0].value, parameters[1].value]
        desc0 = arcpy.Describe(parameters[0].value)
        desc1 = arcpy.Describe(parameters[1].value)
        fc0Name = desc0.name[:-4]
        fc1Name = desc1.name[:-4]
#         Apply intersect operation
        arcpy.Intersect_analysis(in_features,fc0Name + "_" + fc1Name, cluster_tolerance = parameters[2].valueasText , output_type=parameters[3].valueAsText)
        mxd.save()
        return

    def execute(self, parameters, messages):
        self.intersector(parameters)
        return

### Standalone Script Key

The code below is a modified version of the toolbox included above, which can be called via a commandline interface as outlined in Module 3/4. The functionality is entirely the same, however absolute pathes were employed for setting the Map Document and the feature class inputs as parameters. These will need to be modified for each users usecase.

In [ ]:
#  ==========Key: Standalone Script =========
import arcpy
import sys
import time

def dBuffer(fc, bufferD):
    mxd = arcpy.mapping.MapDocument(r"C:\Users\Tannor - Desktop\Documents\ArcEnv\Python for ArcGis\CO_Wells\CO_Wells.mxd")
    df = mxd.activeDataFrame
    default_dir = mxd.filePath[:-12]
    arcpy.env.overwriteOutput = True
    arcpy.Buffer_analysis(fc, default_dir + "\\Buffered_" + fc.rsplit('\\', 1)[-1], str(bufferD) + " Feet", method="GEODESIC")
    arcpy.mapping.AddLayer(df, arcpy.mapping.Layer(default_dir + "\\Buffered_" + fc.rsplit('\\', 1)[-1]))
    return 

def clipper(dirFc,countFc, countName):
    mxd = arcpy.mapping.MapDocument(r"C:\Users\Tannor - Desktop\Documents\ArcEnv\Python for ArcGis\CO_Wells\CO_Wells.mxd")
    default_dir = arcpy.mapping.MapDocument(r"C:\Users\Tannor - Desktop\Documents\ArcEnv\Python for ArcGis\CO_Wells\CO_Wells.mxd").filePath
    default_dir = default_dir.rsplit('\\', 1)[0]
    arcpy.env.overwriteOutput = True
    dirFcName = arcpy.Describe(dirFc).name
    arcpy.MakeFeatureLayer_management(countFc, countName + "_Temp", "COUNTY = 'ARAPAHOE'")
   
    arcpy.MakeFeatureLayer_management(dirFc, "Actual_" + dirFcName, "Dir_status = 'Actual'")
    arcpy.Clip_analysis("Actual_" + dirFcName, countName+"_Temp", arcpy.env.workspace+"\\" + countName + "_Actual_" + dirFcName)
    return

def API10Gen(fc, cField):
    arcpy.env.overwriteOutput = True
    mxd = mxd = arcpy.mapping.MapDocument(r"C:\Users\Tannor - Desktop\Documents\ArcEnv\Python for ArcGis\CO_Wells\CO_Wells.mxd")
    default_dir = mxd.filePath
    default_dir = default_dir.rsplit('\\', 1)[0]

    newFieldName = "API10"
    desc = arcpy.Describe(fc)
    fcName = desc.name
    arcpy.AddField_management(arcpy.env.workspace + "\\" + fcName, newFieldName, "TEXT", "", "", "", "Custom API10", "NON_NULLABLE")
    index = 0     
    APIs = []
    with arcpy.da.SearchCursor(fc, cField) as cursor:
        for row in cursor:
            if(len(row[0]) == 10):
                APIs.append("05" + str(row[0][:-2]))
            else:
                APIs.append("05" + str(row[0]))
            del row
    del cursor
    with arcpy.da.UpdateCursor(fc, newFieldName) as cursor:
        for row in cursor:
            row[0] = APIs[index]
            index += 1
            cursor.updateRow(row)
            del row
    del cursor
    return

def intersector(fc1, fc2, ct, ot):
    mxd = mxd = arcpy.mapping.MapDocument(r"C:\Users\Tannor - Desktop\Documents\ArcEnv\Python for ArcGis\CO_Wells\CO_Wells.mxd")
    arcpy.env.overwriteOutput = True
    default_dir = mxd.filePath
    default_dir = default_dir.rsplit('\\', 1)[0]

    in_features = [fc1, fc2]
    desc0 = arcpy.Describe(fc1)
    desc1 = arcpy.Describe(fc2)
    fc0Name = desc0.name[:-4]
    fc1Name = desc1.name[:-4]
    arcpy.Intersect_analysis(in_features,fc0Name + "_" + fc1Name, cluster_tolerance = ct , output_type=ot)
    return

# Establish the parameters for each method
# Clip parameters:
dirFc = r'C:\Users\Tannor - Desktop\Documents\ArcEnv\Python for ArcGis\CO_Wells\Directional_Lines.shp'
countFc = r'C:\Users\Tannor - Desktop\Documents\ArcEnv\Python for ArcGis\CO_Wells\Colorado_County_Boundaries.shp'
countName = 'Arapahoe'
tstart = time.time()
print("Beginning Clip operation:")
start = time.time()
clipper(dirFc, countFc, countName)
end = time.time()
print("Clip Complete. (" + str(end-start) + ")")
# API10Gen parameters
apiFc = r'C:\Users\Tannor - Desktop\Documents\ArcEnv\Python for ArcGis\CO_Wells\Arapahoe_Actual_Directional_Lines.shp'
cField = r'API'
print("Generating API10s:")
start = time.time()
API10Gen(apiFc, cField)
end = time.time()
print("APIs generated. (" + str(end-start) + ")")
# dBuffer parameters
print("Starting Buffer:")
start = time.time()
dbFc = r"C:\Users\Tannor - Desktop\Documents\ArcEnv\Python for ArcGis\CO_Wells\Arapahoe_Actual_Directional_Lines.shp"
bufferD = 200
dBuffer(dbFc, bufferD)
end = time.time()
print("Buffer Complete. (" + str(end-start) + ")")
# Intersector parameters
print("Starting intersector:")
start = time.time()
fc1 = r"C:\Users\Tannor - Desktop\Documents\ArcEnv\Python for ArcGis\CO_Wells\Buffered_Arapahoe_Actual_Directional_Lines.shp"
fc2 = r"C:\Users\Tannor - Desktop\Documents\ArcEnv\Python for ArcGis\CO_Wells\CO_Quarter_Sections.shp"
ct = '10 Feet'
ot = 'Input'
intersector(fc1, fc2, ct, ot)
end = time.time()
print("Intersection Complete. (" + str(end-start) + ")")
tend = time.time()
print("Script Completed. (" + str(tend-tstart) + ")")
